<a href="https://colab.research.google.com/github/ihshin227/Content_SQL_Sensor/blob/master/Dacon_rain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/dacon')

In [3]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import random
from keras.layers import Dense, Conv2D, Dropout, Conv2DTranspose, MaxPooling2D, BatchNormalization, Activation, concatenate, Input, GlobalAveragePooling2D
from keras import Model
import warnings
from keras.callbacks import ModelCheckpoint
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


In [0]:
warnings.filterwarnings("ignore")

# 재생산성을 위해 시드 고정
np.random.seed(7)
random.seed(7)
tf.random.set_seed(7)

from sklearn.metrics import f1_score

In [0]:
def mae(y_true, y_pred) :
	
	y_true, y_pred = np.array(y_true), np.array(y_pred)
	
	y_true = y_true.reshape(1, -1)[0]
	
	y_pred = y_pred.reshape(1, -1)[0]
	
	over_threshold = y_true >= 0.1
	
	return np.mean(np.abs(y_true[over_threshold] - y_pred[over_threshold]))

def fscore(y_true, y_pred):
	
	y_true, y_pred = np.array(y_true), np.array(y_pred)
	
	y_true = y_true.reshape(1, -1)[0]
	
	y_pred = y_pred.reshape(1, -1)[0]
	
	remove_NAs = y_true >= 0
	
	y_true = np.where(y_true[remove_NAs] >= 0.1, 1, 0)
	
	y_pred = np.where(y_pred[remove_NAs] >= 0.1, 1, 0)
	
	return(f1_score(y_true, y_pred))

def maeOverFscore(y_true, y_pred):
	
	return mae(y_true, y_pred) / (fscore(y_true, y_pred) + 1e-07)

def fscore_keras(y_true, y_pred):
	score = tf.py_function(func=fscore, inp=[y_true, y_pred], Tout=tf.double, name='fscore_keras')
	return score

def maeOverFscore_keras(y_true, y_pred):
	score = tf.py_function(func=maeOverFscore, inp=[y_true, y_pred], Tout=tf.double,  name='custom_mse') 
	return score

In [0]:
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Activation
import keras.backend as backend

def swish(x):
    """Swish activation function.
    # Arguments
        x: Input tensor.
    # Returns
        The Swish activation: `x * sigmoid(x)`.
    # References
        [Searching for Activation Functions](https://arxiv.org/abs/1710.05941)
    """
    if backend.backend() == 'tensorflow':
        try:
            # The native TF implementation has a more
            # memory-efficient gradient implementation
            return backend.tf.nn.swish(x)
        except AttributeError:
            pass

    return x * backend.sigmoid(x)

get_custom_objects().update({'swish': Activation(swish)})

In [0]:
#from keras.utils.training_utils import multi_gpu_model

def fiTest(b, epochs, batch_size, steps_per_epoch):
	#print(b)
	def trainGenerator():

		train_path = './train'
		train_files = sorted(glob.glob(train_path + '/*'))

		for file in train_files:

			dataset = np.load(file)

			target= dataset[:,:,-1].reshape(40,40,1)
			cutoff_labels = np.where(target < 0, 0, target)
			feature = dataset[:,:,b]

			#if (cutoff_labels > 0).sum() < 50:

				#continue

			yield (feature, cutoff_labels)

	train_dataset = tf.data.Dataset.from_generator(trainGenerator, (tf.float32, tf.float32), (tf.TensorShape([40,40,len(b)]),tf.TensorShape([40,40,1])))
	#train_dataset = train_dataset.repeat().batch(2048).prefetch(1)
	#train_dataset = train_dataset.shuffle(batch_size).repeat().batch(batch_size).prefetch(1)
	#train_dataset = train_dataset.repeat().batch(batch_size).prefetch(1)
	train_dataset = train_dataset.repeat().batch(batch_size)
	train_dataset


	test_path = './test'
	test_files = sorted(glob.glob(test_path + '/*'))

	X_test = []

	for file in tqdm(test_files, desc = 'test'):

		data = np.load(file)

		X_test.append(data[:,:,b])

	X_test = np.array(X_test)

	def build_model(input_layer, start_neurons):

		# 40 x 40 -> 20 x 20
		conv1 = Conv2D(start_neurons * 1, (3, 3), activation="swish", padding="same")(input_layer)
		conv1 = Conv2D(start_neurons * 1, (3, 3), activation="swish", padding="same")(conv1)
		pool1 = BatchNormalization()(conv1)
		pool1 = MaxPooling2D((2, 2))(pool1)
		pool1 = Dropout(0.25)(pool1)

		# 20 x 20 -> 10 x 10
		conv2 = Conv2D(start_neurons * 2, (3, 3), activation="swish", padding="same")(pool1)
		conv2 = Conv2D(start_neurons * 2, (3, 3), activation="swish", padding="same")(conv2)
		pool2 = BatchNormalization()(conv2)
		pool2 = MaxPooling2D((2, 2))(pool2)
		pool2 = Dropout(0.25)(pool2)

		# 10 x 10 
		convm = Conv2D(start_neurons * 4, (3, 3), activation="swish", padding="same")(pool2)

		# 10 x 10 -> 20 x 20
		deconv2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(convm)
		uconv2 = concatenate([deconv2, conv2])
		uconv2 = Dropout(0.25)(uconv2)
		uconv2 = Conv2D(start_neurons * 2, (3, 3), activation="swish", padding="same")(uconv2)
		uconv2 = Conv2D(start_neurons * 2, (3, 3), activation="swish", padding="same")(uconv2)
		uconv2 = BatchNormalization()(uconv2)

		# 20 x 20 -> 40 x 40
		deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv2)
		uconv1 = concatenate([deconv1, conv1])
		uconv1 = Dropout(0.25)(uconv1)
		uconv1 = Conv2D(start_neurons * 1, (3, 3), activation="swish", padding="same")(uconv1)
		uconv1 = Conv2D(start_neurons * 1, (3, 3), activation="swish", padding="same")(uconv1)
		uconv1 = BatchNormalization()(uconv1)
		uconv1 = Dropout(0.25)(uconv1)
		output_layer = Conv2D(1, (1,1), padding="same", activation='swish')(uconv1)

		return output_layer

	input_layer = Input((40, 40, len(b)))
	output_layer = build_model(input_layer, 32)
	
	checkpoint = ModelCheckpoint("./model/best_modeltt.hdf5", monitor='maeOverFscore_keras', verbose=1,
	save_best_only=True, mode='auto', period=1)
	
	model = Model(input_layer, output_layer)
	sgd = optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
	#model = multi_gpu_model(model, gpu=4)
	model.compile(loss="mae", optimizer=sgd, metrics=[maeOverFscore_keras, fscore_keras])
	#model.load_weights("./model/best_modeltt.hdf5")

	reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
  
	model_history = model.fit(train_dataset, 
							  epochs = epochs, 
							  verbose=1, 
							  callbacks=[checkpoint, reduce_lr])
	
	loss_history = model_history.history["maeOverFscore_keras"]
	numpy_loss_history = np.array(loss_history)
	np.savetxt("./log/loss_history.txt", numpy_loss_history, delimiter=",")

	pred = model.predict(X_test)
	submission = pd.read_csv('./sample_submission.csv')
	submission.iloc[:,1:] = pred.reshape(-1, 1600)
	submission.to_csv('./output_submission.csv', index = False)

In [8]:
a = np.array([0,1,2,4,5,6,7,8,9,11])
b = a[np.arange(len(a))!=2]

for i in range(10000):
	fiTest(b, 100, 1024, 75)
#fiTest(b, 100, 32, 962*2)

test: 0it [00:00, ?it/s]


AttributeError: ignored

In [0]:
def bn_act(x, act=True):
    x = keras.layers.BatchNormalization()(x)
    if act == True:
        x = keras.layers.Activation("relu")(x)
    return x

def conv_block(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    conv = bn_act(x)
    conv = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides)(conv)
    return conv

def stem(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    conv = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides)(x)
    conv = conv_block(conv, filters, kernel_size=kernel_size, padding=padding, strides=strides)
    
    shortcut = keras.layers.Conv2D(filters, kernel_size=(1, 1), padding=padding, strides=strides)(x)
    shortcut = bn_act(shortcut, act=False)
    
    output = keras.layers.Add()([conv, shortcut])
    return output

def residual_block(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    res = conv_block(x, filters, kernel_size=kernel_size, padding=padding, strides=strides)
    res = conv_block(res, filters, kernel_size=kernel_size, padding=padding, strides=1)
    
    shortcut = keras.layers.Conv2D(filters, kernel_size=(1, 1), padding=padding, strides=strides)(x)
    shortcut = bn_act(shortcut, act=False)
    
    output = keras.layers.Add()([shortcut, res])
    return output

def upsample_concat_block(x, xskip):
    u = keras.layers.UpSampling2D((2, 2))(x)
    c = keras.layers.Concatenate()([u, xskip])
    return c

In [0]:
def ResUNet(input_layer, start_neurons):
    f = [start_neurons, start_neurons*2, start_neurons*4, start_neurons*8, start_neurons*16]
    
    ## Encoder
    e0 = inputs
    e1 = stem(e0, f[0])
    e2 = residual_block(e1, f[1], strides=2)
    e3 = residual_block(e2, f[2], strides=2)
    e4 = residual_block(e3, f[3], strides=2)
    e5 = residual_block(e4, f[4], strides=2)
    
    ## Bridge
    b0 = conv_block(e5, f[4], strides=1)
    b1 = conv_block(b0, f[4], strides=1)
    
    ## Decoder
    u1 = upsample_concat_block(b1, e4)
    d1 = residual_block(u1, f[4])
    
    u2 = upsample_concat_block(d1, e3)
    d2 = residual_block(u2, f[3])
    
    u3 = upsample_concat_block(d2, e2)
    d3 = residual_block(u3, f[2])
    
    u4 = upsample_concat_block(d3, e1)
    d4 = residual_block(u4, f[1])
    
    outputs = keras.layers.Conv2D(1, (1, 1), padding="same", activation="sigmoid")(d4)
    model = keras.models.Model(inputs, outputs)
    return model

In [0]:
tf.__version__

'2.2.0-rc3'